In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
import random

In [2]:
import time

In [3]:
#자동 로그인

#에브리타임 자동 로그인

#가상브라우저 사용
options = wd.ChromeOptions()
#options.add_argument("no-sandbox")
#options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent={'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36}")

# 크롬드라이버를 다운받은 경로를 넣기
driver = wd.Chrome(executable_path='C:\selenium_server/chromedriver', options = options )
#driver.implcitly_wait(1)


baseURL = "https://everytime.kr/login"

rand_value = random.uniform(2,4) #랜덤한 시간으로
time.sleep(rand_value) #sleep 쉬어주기

driver.get(baseURL)

rand_value = random.uniform(2,4)
time.sleep(rand_value)

driver.find_element_by_name('userid').send_keys() # 괄호 안에 id
driver.find_element_by_name('password').send_keys() # 괄호 안에 password

rand_value = random.uniform(2,4)
time.sleep(rand_value)
driver.find_element_by_xpath('//*[@class="submit"]/input').click()



In [4]:
# dictionary {'교양 카테고리' :[(강의명, 교수님 성함),(강의명, 교수님 성함),...]}
lecture_info = {
 
}
   

In [7]:
data = pd.DataFrame()
for key in lecture_info.keys():
    rand_value = random.uniform(1,3)
    time.sleep(rand_value)
    for lec, prof in lecture_info[key]:
        # 강의평 화면으로 넘어가기
        driver.get('https://everytime.kr/lecture/search/' + lec)

        
        # 검색하고 나서 긁어오고, 강의평 들어가는 링크 찾기
        rand_value = random.uniform(2,4)
        time.sleep(rand_value)
        html = driver.page_source
        soup = bs(html,'html.parser')


        # 여러 강의 중 특정 교수님의 강의 찾기
        lectures = soup.find_all('a', class_ = 'lecture')
        for lecture in lectures:
            if lecture.find('p', class_='professor').get_text() == prof:
                lecture_url = lecture['href']
                break
        rand_value = random.uniform(1,2)
        time.sleep(rand_value)
        #soup.find_all('p',class_='professor')
        
        
        driver.get("https://everytime.kr/" + lecture_url)
        rand_value = random.uniform(3,4)
        time.sleep(rand_value)


        # 강의평 링크에서 page_source 읽어오기
        html = driver.page_source
        soup = bs(html,'html.parser')


        # 강의평 정보 수집
        articles = soup.find_all('article')

        stars = []
        semesters = []
        reviews = []


        for article in articles:
            if article.find('div', class_ ='pay') == None: # 족보나 시험 후기 div.article 제외 
                star = article.find('span', class_ = 'on')['style']
                review = article.find('p', class_ = 'text').get_text()
                
                if article.find('span', class_ = 'semester') is not None: # semester 정보가 없는 경우 고려
                    semester = article.find('span', class_ = 'semester').get_text()
                else:
                    semester = None

                stars.append(star)
                semesters.append(semester)
                reviews.append(review)
            else:
                continue

        temp_df = pd.DataFrame({
            'category' : key,
            'lecture' : lec,
            'professor' : prof,
            'review' : reviews,
            'stars' : stars,
            'semester' : semesters
        })
        
        data = data.append(temp_df, ignore_index = True)

46
45
46
45


KeyboardInterrupt: 

In [32]:
data.to_csv('everytime_reviews_.csv', index = False)